<h1>GraphX Assignment</h1>
In this assignment, you need to do the following:
<li>Read the file 201710-citibike-tripdata.csv</li>
<li>Construct a graph with stations as vertices and trips between stations as edges</li>
<li>Vertex Ids are station numbers and Vertex attributes are station names</li>
<li>Edge attributes are trip duration (the first field in the file - durations are in seconds)</li>
<li>Then answer the questions below</li>

In [1]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.14:4041
SparkContext available as 'sc' (version = 2.3.3, master = local[*], app id = local-1555991114350)
SparkSession available as 'spark'


import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD


In [2]:
val text = sc.textFile("C://Users//vaish//Downloads//201710-citibike-tripdata (1).csv")
val text_header_removed = text.mapPartitionsWithIndex{ (idx,iter) => if (idx==0) iter.drop(1) else iter}
val vertexRDD = (text_header_removed.map(l => ((l.split(",")(3).toLong,(l.split(",")(4))))).
                 union(text_header_removed.map(l => ((l.split(",")(7).toLong,(l.split(",")(8)))))))
val edgeRDD = text_header_removed.map(l=> (Edge(l.split(",")(3).toLong, l.split(",")(7).toLong, l.split(",")(0).toInt)))
val graph: Graph[(String), Int] = Graph(vertexRDD, edgeRDD)
graph.partitionBy(PartitionStrategy.CanonicalRandomVertexCut)

text: org.apache.spark.rdd.RDD[String] = C://Users//vaish//Downloads//201710-citibike-tripdata (1).csv MapPartitionsRDD[1] at textFile at <console>:29
text_header_removed: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at mapPartitionsWithIndex at <console>:30
vertexRDD: org.apache.spark.rdd.RDD[(Long, String)] = UnionRDD[5] at union at <console>:32
edgeRDD: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[6] at map at <console>:33
graph: org.apache.spark.graphx.Graph[String,Int] = org.apache.spark.graphx.impl.GraphImpl@d2e9567
res0: org.apache.spark.graphx.Graph[String,Int] = org.apache.spark.graphx.impl.GraphImpl@23b6274d


<h2>Station from which most trips originate</h2>
<li>Print the name of the station from which most trips originate</li>

In [3]:
val most = graph.outDegrees.join(vertexRDD).distinct.sortBy(_._2._1, ascending=false).first._2._2

most: String = "Pershing Square North"


In [4]:
println(s"The station with the most originating trips is ${most}")

The station with the most originating trips is "Pershing Square North"


<h2>Proportion of trips for each station that start and end at that same station</h2>
<li>Use aggregateMessages to calculate the number of trips that start and end at the same vertex (for each vertex)</li>
<li>Divide that number by the number of trips that originate at that vertex using a join with outDegrees</li>
<li>Note that some stations may have no trips at all (i.e., the denominator may be non-existent)</li>
<li>Print the names of the top ten stations with the highest proportions</li>

In [5]:
val start_end_agg = graph.aggregateMessages[Int](ec => if (ec.srcId == ec.dstId) {ec.sendToSrc(1)}, (x,y) => x+y)

start_end_agg: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[50] at RDD at VertexRDD.scala:57


In [6]:
val prop_map = start_end_agg.fullOuterJoin(graph.outDegrees).map(t => (t._1.toLong, t._2 match {
    case ((Some(b), Some(c))) => (b.toDouble/c.toDouble)
    case ((Some(b), None)) => 0
    case ((None, Some(c))) => 0
    case ((None, None)) => 0
}))                                              

prop_map: org.apache.spark.rdd.RDD[(Long, Double)] = MapPartitionsRDD[54] at map at <console>:37


In [7]:
prop_map.map(t => ((t._1), t._2)).first

res2: (Long, Double) = (3454,0.014084507042253521)


Names of stations with highest proportions 

In [8]:
val top10 = vertexRDD.join(prop_map).distinct.sortBy(_._2._2, ascending=false).map(t =>
    t._2._1 + " - " + t._2._2).take(10).foreach(println)

"8D QC Station 01" - 1.0
"NYCBS DEPOT - DELANCEY" - 0.5
"Yankee Ferry Terminal" - 0.3433333333333333
"Soissons Landing" - 0.32545454545454544
"Pioneer St & Richards St" - 0.19732441471571907
"39 St & 2 Ave - Citi Bike HQ at Industry City" - 0.1836734693877551
"Ditmars Blvd & 19 St" - 0.171990171990172
"Brooklyn Bridge Park - Pier 2" - 0.17134416543574593
"West Drive & Prospect Park West" - 0.14900527811611855
"Expansion Warehouse 333 Johnson Ave" - 0.125


top10: Unit = ()


<h2>Create a new graph that contains all edges except for those between the same station</h2>
<li>Report the number of edges in the new graph

In [9]:
//val new_edge_RDD = graph.edges.filter(t => t.srcId != t.dstId)
//val new_graph = (vertexRDD, new_edge_RDD)
val new_graph: Graph[(String), Int] = Graph(vertexRDD, graph.edges.filter(t => t.srcId != t.dstId))
new_graph.edges.count

new_graph: org.apache.spark.graphx.Graph[String,Int] = org.apache.spark.graphx.impl.GraphImpl@6891e514
res3: Long = 1864347


<h2>Calculate the average time between every pair of stations</h2>
<li>Use groupEdges to calculate the total time between each pair of stations</li>
<li>Get rdd of count of number of edges by using reduceByKey on graph.edges</li>
<li>get edges from groupEdges in the same pair format as the count of number of edges</li>
<li>join the two rdds, use map to calculate averages, construct new edges</li>
<li>make a new graph with vertices from total_times and the new edges you've just constructed</li>
<li>Print the top 10 edges with the highest average time between stations<li>
    
Sample Output:
<pre>
From: "Fulton St & Utica Ave"	To: "Crown St & Bedford Ave"	Average Time: 9340261.0
From: "Verona Pl & Fulton St"	To: "Humboldt St & Varet St"	Average Time: 9163224.0
From: "Albany Ave & Fulton St"	To: "Sullivan Pl & Bedford Ave"	Average Time: 8461569.0
From: "S Portland Ave & Hanson Pl"	To: "NYCBS DEPOT - DELANCEY"	Average Time: 5967124.0
From: "Greenwich Ave & Charles St"	To: "NYCBS Depot - GOW"	Average Time: 5954675.0
From: "Myrtle Ave & Marcy Ave"	To: "NYCBS Depot - GOW"	Average Time: 5759985.0
From: "Macon St & Nostrand Ave"	To: "NYCBS Depot - GOW"	Average Time: 5296261.5
From: "Lewis Ave & Madison St"	To: "NYCBS DEPOT - DELANCEY"	Average Time: 4836816.0
From: "Albany Ave & Fulton St"	To: "NYCBS DEPOT - DELANCEY"	Average Time: 4316721.0
From: "Union Ave & Wallabout St"	To: "NYCBS Depot - GOW"	Average Time: 4218267.0
</pre>


In [10]:
val total_time = graph.partitionBy(PartitionStrategy.CanonicalRandomVertexCut)
.groupEdges((edge1, edge2) => edge1 + edge2).triplets.sortBy(_.attr, ascending=false)
.map(triplet =>((triplet.srcId.toLong, triplet.dstId.toLong), triplet.attr))

val total_trips = graph.edges.map(a => ((a.srcId.toLong, a.dstId.toLong),1)).reduceByKey((a,b)=> a+b)

total_time: org.apache.spark.rdd.RDD[((Long, Long), Int)] = MapPartitionsRDD[103] at map at <console>:37
total_trips: org.apache.spark.rdd.RDD[((Long, Long), Int)] = ShuffledRDD[105] at reduceByKey at <console>:39


In [11]:
val avg_time_edges = total_time.join(total_trips).map(t => Edge(t._1._1, t._1._2, (t._2._1/t._2._2).toDouble))
val avg_time_graph: Graph[String, Double] = Graph(vertexRDD, avg_time_edges)

avg_time_edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Double]] = MapPartitionsRDD[109] at map at <console>:38
avg_time_graph: org.apache.spark.graphx.Graph[String,Double] = org.apache.spark.graphx.impl.GraphImpl@3e25ca72


In [12]:
avg_time_graph.triplets.sortBy(_.attr, ascending=false)
  .map(triplet =>
    "From: " + triplet.srcAttr + " To " + triplet.dstAttr + " Average Time: " + triplet.attr).take(10).
foreach(println)

From: "Fulton St & Utica Ave" To "Crown St & Bedford Ave" Average Time: 9340261.0
From: "Verona Pl & Fulton St" To "Humboldt St & Varet St" Average Time: 9163224.0
From: "Albany Ave & Fulton St" To "Sullivan Pl & Bedford Ave" Average Time: 8461569.0
From: "S Portland Ave & Hanson Pl" To "NYCBS DEPOT - DELANCEY" Average Time: 5967124.0
From: "Greenwich Ave & Charles St" To "NYCBS Depot - GOW" Average Time: 5954675.0
From: "Myrtle Ave & Marcy Ave" To "NYCBS Depot - GOW" Average Time: 5759985.0
From: "Macon St & Nostrand Ave" To "NYCBS Depot - GOW" Average Time: 5296261.0
From: "Lewis Ave & Madison St" To "NYCBS DEPOT - DELANCEY" Average Time: 4836816.0
From: "Albany Ave & Fulton St" To "NYCBS DEPOT - DELANCEY" Average Time: 4316721.0
From: "Union Ave & Wallabout St" To "NYCBS Depot - GOW" Average Time: 4218267.0


<h2>Important stations</h2>
Citibike wants to figure out how best to deploy its workers in checking whether a station is over-full (too many bikes) or needs more bikes. It figures that the best way to do this is to find out which stations are the most important in terms of flows:
<li>A station that has high bike returns and is connected to other stations with high bike returns is more likely to have too many bikes in its station and therefore should be monitored more often</li>
<li>A station that has high bike pickups and is connected to other stations with high bike pickups is more likely to be short of bikes and therefore should be monitored more often</li>
<li>Calculate the propensities for over-fullness and emptiness for every station</li>
<li>Report the 10 most important stations for over-fullness</li>
<li>Report the 10 most important stations for emptiness</li>

Sample inflow importance:

<pre>
Station: "Pershing Square North"	Importance: 4.930887390071603
Station: "West St & Chambers St"	Importance: 3.7410934274030576
Station: "Broadway & E 22 St"	Importance: 3.58520147183096
Station: "E 17 St & Broadway"	Importance: 3.537658018512581
Station: "W 21 St & 6 Ave"	Importance: 3.438585855241344
Station: "8 Ave & W 31 St"	Importance: 3.338364421207131
Station: "Centre St & Chambers St"	Importance: 3.15384598074596
Station: "Broadway & E 14 St"	Importance: 3.0325702821942904
Station: "12 Ave & W 40 St"	Importance: 2.9542359558471127
Station: "W 20 St & 11 Ave"	Importance: 2.8726816010849965
</pre>

Sample outflow importance:

<pre>
Station: "Hs Don't Use"	Importance: 5.7106408695207485
Station: "Pershing Square North"	Importance: 5.012823444592197
Station: "WS Don't Use"	Importance: 4.272284643284594
Station: "Broadway & E 22 St"	Importance: 3.451521106903819
Station: "E 17 St & Broadway"	Importance: 3.334725974545745
Station: "W 21 St & 6 Ave"	Importance: 3.2837275306670706
Station: "Grand Army Plaza & Central Park S"	Importance: 3.269839753653885
Station: "West St & Chambers St"	Importance: 3.2398990476596206
Station: "8 Ave & W 31 St"	Importance: 3.2023851500938387
Station: "Central Park S & 6 Ave"	Importance: 3.048749328362056
</pre>

In [19]:
val a = graph.pageRank(0.0001).vertices
val rank = sc.parallelize(a.sortBy(_._2,ascending=false).take(10))
val station = graph.vertices
val inTop10 = station.join(rank).map{
    case (id,(name,r)) => (name,r)
}

a: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[2712] at RDD at VertexRDD.scala:57
rank: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Double)] = ParallelCollectionRDD[2724] at parallelize at <console>:43
station: org.apache.spark.graphx.VertexRDD[String] = VertexRDDImpl[16] at RDD at VertexRDD.scala:57
inTop10: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[2728] at map at <console>:45


In [20]:
//Print the output here
inTop10.sortBy(_._2,ascending=false).map(b=>s"Station: ${b._1}  Importance: ${b._2}   ").collect().take(10).foreach(println)

Station: "Pershing Square North"  Importance: 4.930887390071603   
Station: "West St & Chambers St"  Importance: 3.7410934274030576   
Station: "Broadway & E 22 St"  Importance: 3.58520147183096   
Station: "E 17 St & Broadway"  Importance: 3.537658018512581   
Station: "W 21 St & 6 Ave"  Importance: 3.438585855241344   
Station: "8 Ave & W 31 St"  Importance: 3.338364421207131   
Station: "Centre St & Chambers St"  Importance: 3.15384598074596   
Station: "Broadway & E 14 St"  Importance: 3.0325702821942904   
Station: "12 Ave & W 40 St"  Importance: 2.9542359558471127   
Station: "W 20 St & 11 Ave"  Importance: 2.8726816010849965   


In [16]:
val edge = graph.edges.map(e=>Edge(e.dstId,e.srcId,e.attr))
val new_graph = Graph(graph.vertices,edge)
val rank = new_graph.pageRank(0.0001).vertices
val o_rank = sc.parallelize(rank.sortBy(_._2,ascending=false).take(10))
val station = graph.vertices
val outTop10 = station.join(o_rank).map{
    case (a,(b,c)) => (b,c)
}

2019-04-22 23:49:29 WARN  BlockManager:66 - Asked to remove block rdd_1819_5, which does not exist
2019-04-22 23:49:29 WARN  BlockManager:66 - Asked to remove block rdd_1819_6, which does not exist
2019-04-22 23:49:29 WARN  BlockManager:66 - Asked to remove block rdd_1819_11, which does not exist
2019-04-22 23:49:29 WARN  BlockManager:66 - Asked to remove block rdd_1819_17, which does not exist


edge: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[993] at map at <console>:40
new_graph: org.apache.spark.graphx.Graph[String,Int] = org.apache.spark.graphx.impl.GraphImpl@1b881bdf
rank: org.apache.spark.graphx.VertexRDD[Double] = VertexRDDImpl[1846] at RDD at VertexRDD.scala:57
o_rank: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, Double)] = ParallelCollectionRDD[1858] at parallelize at <console>:43
station: org.apache.spark.graphx.VertexRDD[String] = VertexRDDImpl[16] at RDD at VertexRDD.scala:57
outTop10: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[1862] at map at <console>:45


In [18]:
//Print the output here
outTop10.sortBy(_._2,ascending=false).map(b=>s"Station: ${b._1}  Importance: ${b._2}   ").collect().take(10).foreach(println)

Station: "Hs Don't Use"  Importance: 5.710640869520747   
Station: "Pershing Square North"  Importance: 5.012823444592195   
Station: "WS Don't Use"  Importance: 4.272284643284593   
Station: "Broadway & E 22 St"  Importance: 3.4515211069038183   
Station: "E 17 St & Broadway"  Importance: 3.3347259745457443   
Station: "W 21 St & 6 Ave"  Importance: 3.2837275306670697   
Station: "Grand Army Plaza & Central Park S"  Importance: 3.269839753653884   
Station: "West St & Chambers St"  Importance: 3.2398990476596197   
Station: "8 Ave & W 31 St"  Importance: 3.2023851500938383   
Station: "Central Park S & 6 Ave"  Importance: 3.0487493283620557   
